## Linear Regression Example

In [68]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [69]:
# 参数
learning_rate = 0.01
training_epochs = 1000
display_step = 50

In [70]:
# 训练数据
train_X = np.asarray([3.3,4.4,5.5,6.71,6.93,4.168,9.779,6.182,7.59,2.167,
                         7.042,10.791,5.313,7.997,5.654,9.27,3.1])
train_Y = np.asarray([1.7,2.76,2.09,3.19,1.694,1.573,3.366,2.596,2.53,1.221,
                         2.827,3.465,1.65,2.904,2.42,2.94,1.3])
n_samples = train_X.shape[0]

In [71]:
# tf输入图形
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

In [72]:
# 设置模型权重和偏置项
W = tf.Variable(np.random.randn(), name="weight")
b = tf.Variable(np.random.randn(), name="bias")

In [73]:
pred = tf.add(tf.multiply(X, W), b)

In [74]:
# 均方误差，误差的和 reduce_sum 
cost = tf.reduce_sum(tf.pow(y - pred, 2)) / (2 * n_samples)

# 梯度下降优化
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)

In [75]:
# 初始化变量（比如赋予初始值)
init = tf.global_variables_initializer()

In [79]:
# Start training
with tf.Session() as sess:
    sess.run(init)
    # Fit all training data
    for epoch in range(training_epochs):
        for (x, y) in zip(train_X, train_Y):
            sess.run(optimizer, feed_dict={X:x , Y: y})
        if (epoch + 1 % display_step == 0):
            print("epoch")
            c = sess.run(cost, feed_dict={X:train_X, y:train_Y})
            print("Epoch:", "%04d" % (epoch + 1), "cost=", "{:.9f}".format(c), "W=", 
                      sess.run(W), "b=", sess.run(b))